# Create  Spark Context 

In [1]:
import pyspark
from pyspark.sql import SQLContext

In [84]:
try : 
    sc = pyspark.SparkContext('local[*]')
except Exception as e : 
    print( "sc already exists ")
sqlContext = SQLContext(sc)
print("done")

done


# Let's see SC works fine 

In [83]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)
print(rdd.collect())
sc.stop()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

# Collect Data from Web REST API 
- url : https://openweathermap.org/forecast5 <br>
※ Let's check out how the data looks like 

온도, 파도, 기압, 습도 등등 을 가지고 결국 기상청에서 예보한 기상 정보를 카테고리를 예측하는 모델을 만들기 위해서 기존에 조금 복잡한 구조의 Json 데이터에서 훈련이 용이하도록 편집한다. 

In [89]:
import requests
from bs4 import BeautifulSoup
import re, json, os, random
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce 
import pandas as pd
from io import StringIO
from sklearn.cross_validation import train_test_split

resp = requests.get('http://openweathermap.org/data/2.5/forecast?q=London,us&mode=json&appid=b1b15e88fa797225412429c1c50c122a1')
data = resp.json()

x_data = []
y_data = []
data_list = data['list']
for raw in data_list :
    x_data.append(raw['main'])
    y_data.append(raw['weather'][0]['main'])

print("x_data : {0}".format(x_data[0:2]))
print("y_data : {0}".format(y_data[0:2]))

x_data : [{'pressure': 1011.5, 'grnd_level': 1011.5, 'temp_kf': 2.55, 'temp': 25.82, 'temp_max': 25.82, 'temp_min': 23.27, 'sea_level': 1027.32, 'humidity': 57}, {'pressure': 1010.24, 'grnd_level': 1010.24, 'temp_kf': 1.91, 'temp': 27.5, 'temp_max': 27.5, 'temp_min': 25.59, 'sea_level': 1026.07, 'humidity': 49}]
y_data : ['Clear', 'Clear']


# Train /Test 데이터 분리 
정확한 테스트를 위해 Train 데이터와 Test 데이터를 분리한다 .

In [93]:
data_train, data_test, labels_train, labels_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)
def change (x) :
    if(x == 'Clear') :
        return 0
    if(x in ['Rain', 'Clouds']) :
        return 1
#     idx = ['Clear', 'Rain', 'Clouds']
#     return idx.index(x)

labels_train = list(map(lambda x : change(x), labels_train ))
labels_test = list(map(lambda x : change(x), labels_test ))

print("data_train : {0}".format(data_train[0:5]))
print("data_test : {0}".format(data_test[0:5]))
print("labels_train : {0}".format(labels_train[0:5]))
print("labels_test : {0}".format(labels_test[0:5]))

data_train : [{'pressure': 1016.84, 'grnd_level': 1016.84, 'temp_kf': 0, 'temp': 20.29, 'temp_max': 20.29, 'temp_min': 20.29, 'sea_level': 1032.73, 'humidity': 66}, {'pressure': 1015.57, 'grnd_level': 1015.57, 'temp_kf': 0, 'temp': 19.37, 'temp_max': 19.37, 'temp_min': 19.37, 'sea_level': 1031.56, 'humidity': 76}, {'pressure': 1009.6, 'grnd_level': 1009.6, 'temp_kf': 0, 'temp': 23.19, 'temp_max': 23.19, 'temp_min': 23.19, 'sea_level': 1025.51, 'humidity': 62}, {'pressure': 1014.58, 'grnd_level': 1014.58, 'temp_kf': 0, 'temp': 15.14, 'temp_max': 15.14, 'temp_min': 15.14, 'sea_level': 1030.62, 'humidity': 88}, {'pressure': 1012.83, 'grnd_level': 1012.83, 'temp_kf': 0, 'temp': 14.99, 'temp_max': 14.99, 'temp_min': 14.99, 'sea_level': 1028.85, 'humidity': 99}]
data_test : [{'pressure': 1015.77, 'grnd_level': 1015.77, 'temp_kf': 0, 'temp': 21.1, 'temp_max': 21.1, 'temp_min': 21.1, 'sea_level': 1031.67, 'humidity': 71}, {'pressure': 1013.25, 'grnd_level': 1013.25, 'temp_kf': 0, 'temp': 14.35

# Save Data as file

In [94]:
with open('/home/jovyan/work/train6.data', 'w') as fp : 
    for data, label in zip(data_train, labels_train) :
        x_data = []
        x_data.append(label)
        for raw in data.keys() :
            x_data.append(data[raw])
        x_data.append('\n')
        #train_feed_data.append(LabeledPoint(label, x_data))
        fp.write(' '.join(map(lambda x: str(x) ,x_data)))
               

# Train Model

In [95]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

# 파일이 있는지 확인 
# with open('/home/jovyan/work/train5.data', 'r') as fp : 
#     print(fp.read())
    
data = sc.textFile("file:///home/jovyan/work/train6.data")
#print(data.collect())

In [128]:
def parsePoint(line):
    values = [float(x) for x in line.split(' ')[0:len(line.split(' '))-1]] 
    return LabeledPoint(values[0], values[1:])

parsedData = data.map(parsePoint)
model = LogisticRegressionWithLBFGS.train(parsedData)
print(model)

(weights=[-0.81230598677,-0.81230598677,-100.271879002,-0.603157535245,-0.603157535245,1.97910338392,1.55627691027,0.415274547681], intercept=0.0)


# Test Model
Test Model Accuracy  

In [132]:
# print("labels_train : {0}".format(labels_train[0:5]))
# print("labels_test : {0}".format(labels_test[0:5]))
# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda x: x[0] != x[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.13793103448275862


# Save Model

In [125]:
import random 
model.save(sc, "/home/jovyan/work/" + str(random.randrange(1,50)) + '/')
print("save model")

save model
